In [34]:
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# Simulated Annealing
import yfinance as yf


In [35]:
df_original = yf.Tickers('NQ=F')
df_original = df_original.history(start='2010-01-01', end='2024-01-01')

[*********************100%***********************]  1 of 1 completed


In [36]:
df_original.columns

MultiIndex([(       'Close', 'NQ=F'),
            (   'Dividends', 'NQ=F'),
            (        'High', 'NQ=F'),
            (         'Low', 'NQ=F'),
            (        'Open', 'NQ=F'),
            ('Stock Splits', 'NQ=F'),
            (      'Volume', 'NQ=F')],
           names=['Price', 'Ticker'])

### Función de backtest para cada combinación de parámetros


In [37]:
def backtest_strategy(sma_1, sma_2):
    df = df_original.copy()
    df['SMA_50'] = df['Close'].rolling(window=sma_1).mean()
    df['SMA_200'] = df['Close'].rolling(window=sma_2).mean()

    df['Signal'] = 0
    df.loc[df['SMA_50'] > df['SMA_200'], 'Signal'] = 1
    df.loc[df['SMA_50'] < df['SMA_200'], 'Signal'] = -1

    df['Position'] = df['Signal'].shift()
    df['Strategy_Returns'] = df['Position'] * df['Close'].squeeze().pct_change()

    df.dropna(inplace=True)

    # Rendimiento total de la estrategia
    return (df['Strategy_Returns'] + 1).prod() - 1

### Función de aceptación de Simulated Annealing

In [38]:
def acceptance_probability(old_cost, new_cost, temperature):
    if new_cost > old_cost:
        return 1.0
    else:
        return np.exp((new_cost - old_cost) / temperature)

### Función de Simulated Annealing


In [39]:
def simulated_annealing(initial_sma_1, initial_sma_2, initial_temp, cooling_rate, max_iter):
    current_sma_1 = initial_sma_1
    current_sma_2 = initial_sma_2
    current_cost = backtest_strategy(current_sma_1, current_sma_2)
    best_sma_1, best_sma_2 = current_sma_1, current_sma_2
    best_cost = current_cost
    temp = initial_temp

    for i in range(max_iter):
        # Generar nuevos parámetros vecinos (cercanos a los actuales)
        new_sma_1 = current_sma_1 + random.randint(-5, 5)
        new_sma_2 = current_sma_2 + random.randint(-5, 5)

        # Asegurarse de que los parámetros tengan sentido (evitar valores negativos o cruces imposibles)
        new_sma_1 = max(10, new_sma_1)
        new_sma_2 = max(20, new_sma_2)
        new_sma_2 = max(new_sma_1 + 10, new_sma_2)  # Asegurar que SMA_2 sea mayor que SMA_1

        # Calcular el nuevo costo (rendimiento) con los nuevos parámetros
        new_cost = backtest_strategy(new_sma_1, new_sma_2)

        # Decidir si aceptamos la nueva solución o no
        if acceptance_probability(current_cost, new_cost, temp) > random.random():
            current_sma_1 = new_sma_1
            current_sma_2 = new_sma_2
            current_cost = new_cost

        # Actualizar la mejor solución encontrada
        if new_cost > best_cost:
            best_sma_1, best_sma_2 = new_sma_1, new_sma_2
            best_cost = new_cost

        # Enfriar la temperatura
        temp *= cooling_rate

        print(f'Iteración {i+1}: Mejor SMA_1 = {best_sma_1}, Mejor SMA_2 = {best_sma_2}, Mejor Rendimiento = {best_cost:.2%}')
    
    return best_sma_1, best_sma_2, best_cost


### Parámetros iniciales para Simulated Annealing


In [40]:
initial_sma_1 = 100
initial_sma_2 = 300
initial_temp = 1000
cooling_rate = 0.95
max_iter = 100

### Ejecutar el algoritmo de Simulated Annealing


In [41]:
best_sma_1, best_sma_2, best_return = simulated_annealing(initial_sma_1, initial_sma_2, 
                                                          initial_temp, cooling_rate, max_iter)

Iteración 1: Mejor SMA_1 = 100, Mejor SMA_2 = 300, Mejor Rendimiento = 310.68%
Iteración 2: Mejor SMA_1 = 100, Mejor SMA_2 = 304, Mejor Rendimiento = 314.83%
Iteración 3: Mejor SMA_1 = 100, Mejor SMA_2 = 304, Mejor Rendimiento = 314.83%
Iteración 4: Mejor SMA_1 = 100, Mejor SMA_2 = 304, Mejor Rendimiento = 314.83%
Iteración 5: Mejor SMA_1 = 100, Mejor SMA_2 = 304, Mejor Rendimiento = 314.83%
Iteración 6: Mejor SMA_1 = 100, Mejor SMA_2 = 304, Mejor Rendimiento = 314.83%
Iteración 7: Mejor SMA_1 = 100, Mejor SMA_2 = 304, Mejor Rendimiento = 314.83%
Iteración 8: Mejor SMA_1 = 100, Mejor SMA_2 = 304, Mejor Rendimiento = 314.83%
Iteración 9: Mejor SMA_1 = 100, Mejor SMA_2 = 304, Mejor Rendimiento = 314.83%


Iteración 10: Mejor SMA_1 = 100, Mejor SMA_2 = 304, Mejor Rendimiento = 314.83%
Iteración 11: Mejor SMA_1 = 100, Mejor SMA_2 = 304, Mejor Rendimiento = 314.83%
Iteración 12: Mejor SMA_1 = 100, Mejor SMA_2 = 304, Mejor Rendimiento = 314.83%
Iteración 13: Mejor SMA_1 = 100, Mejor SMA_2 = 304, Mejor Rendimiento = 314.83%
Iteración 14: Mejor SMA_1 = 100, Mejor SMA_2 = 304, Mejor Rendimiento = 314.83%
Iteración 15: Mejor SMA_1 = 100, Mejor SMA_2 = 304, Mejor Rendimiento = 314.83%
Iteración 16: Mejor SMA_1 = 100, Mejor SMA_2 = 304, Mejor Rendimiento = 314.83%
Iteración 17: Mejor SMA_1 = 100, Mejor SMA_2 = 304, Mejor Rendimiento = 314.83%
Iteración 18: Mejor SMA_1 = 100, Mejor SMA_2 = 304, Mejor Rendimiento = 314.83%
Iteración 19: Mejor SMA_1 = 100, Mejor SMA_2 = 304, Mejor Rendimiento = 314.83%
Iteración 20: Mejor SMA_1 = 100, Mejor SMA_2 = 304, Mejor Rendimiento = 314.83%
Iteración 21: Mejor SMA_1 = 100, Mejor SMA_2 = 304, Mejor Rendimiento = 314.83%
Iteración 22: Mejor SMA_1 = 100, Mejor S

In [43]:
print(f'Los mejores parámetros son SMA_1: {best_sma_1}, SMA_2: {best_sma_2} con un rendimiento total de {best_return:.2%}')


Los mejores parámetros son SMA_1: 100, SMA_2: 304 con un rendimiento total de 314.83%
